In [48]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload
import io

def download_file_from_drive(file_id, credentials_file, save_path):
    """Downloads a file from Google Drive using a service account."""

    SCOPES = ['https://www.googleapis.com/auth/drive.readonly'] 

    creds = service_account.Credentials.from_service_account_file(
        credentials_file, scopes=SCOPES)

    try:
        service = build('drive', 'v3', credentials=creds)

        # Get the file metadata
        file_metadata = service.files().get(fileId=file_id).execute()
        file_name = file_metadata.get('name')
        mime_type = file_metadata.get('mimeType')

        # Download the file
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}%.")

        # Save the file
        with open(save_path + file_name, 'wb') as f:
            f.write(fh.getvalue())

        print(f"File '{file_name}' downloaded successfully to '{save_path}'.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
file_id = '1qFFZXFNsVLargf5AtRCbR0vPeQRjeoz8'  # Replace with the file ID from Google Drive
credentials_file = '/Users/muthia_ap/Documents/find_my_way/Baskit/config/elevated-valve-454202-n0-bf831559f8c6.json' # replace with your credentials file name
save_path = '/Users/muthia_ap/Documents/find_my_way/Baskit/data/'  # Replace with the desired save path

download_file_from_drive(file_id, credentials_file, save_path)

Download 100%.
File 'Take Home Test Data Engineer Baskit.xlsx' downloaded successfully to '/Users/muthia_ap/Documents/find_my_way/Baskit/data/'.


In [57]:
import gspread
import pandas as pd
import sqlite3
import io
from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload
from oauth2client.service_account import ServiceAccountCredentials

In [58]:
def download_file_from_drive(file_id, credentials_file, save_path):
    """Downloads a file from Google Drive using a service account."""
    SCOPES = ['https://www.googleapis.com/auth/drive.readonly'] 
    creds = service_account.Credentials.from_service_account_file(
        credentials_file, scopes=SCOPES)
    
    try:
        service = build('drive', 'v3', credentials=creds)
        
        # Get the file metadata
        file_metadata = service.files().get(fileId=file_id).execute()
        file_name = file_metadata.get('name')
        
        # Download the file
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}%.")

        # Save the file
        full_path = save_path + file_name
        with open(full_path, 'wb') as f:
            f.write(fh.getvalue())
        
        print(f"File '{file_name}' downloaded successfully to '{save_path}'.")
        return full_path
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [59]:
def extract_from_google_sheets(sheet_path, sheet_name):
    """Extract data from a local Excel file"""
    try:
        df = pd.read_excel(sheet_path, sheet_name=sheet_name, skiprows=1)
        return df
    except Exception as e:
        print(f"An error occurred while reading the Excel file: {e}")
        return None

In [60]:
def transform_data(df):
    """Transform the extracted data"""
    # Rename columns to snake_case
    df.rename(columns={"born_day": "born_date"}, inplace=True)
    
    # Standardize date format
    df["born_date"] = pd.to_datetime(df["born_date"], errors='coerce').dt.strftime("%Y-%m-%d")
    
    # Standardize phone number format
    df["phone_number"] = df["phone_number"].astype(str).apply(
        lambda x: "+62" + x.lstrip("0") if not x.startswith("62") else "+" + x)
    
    # Remove duplicates
    df.drop_duplicates(inplace=True)
    
    return df

In [61]:
def load_to_sqlite(df, db_path, table_name):
    """Load the transformed data into SQLite"""
    conn = sqlite3.connect(db_path)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()


In [62]:
def main():
    """Main ETL Process"""
    SHEET_NAME = "data"  # Adjust if different
    CREDS_PATH = "/Users/muthia_ap/Documents/find_my_way/Baskit/config/elevated-valve-454202-n0-bf831559f8c6.json"  # Update with your actual path
    DB_PATH = "etl_data.db"
    TABLE_NAME = "born_date_data"
    DRIVE_FILE_ID = "1qFFZXFNsVLargf5AtRCbR0vPeQRjeoz8"  # Add your Google Drive file ID
    SAVE_PATH = "./data/"  # Path where the file will be saved
    
    print("Downloading file from Google Drive...")
    downloaded_file = download_file_from_drive(DRIVE_FILE_ID, CREDS_PATH, SAVE_PATH)
    if not downloaded_file:
        print("Failed to download the file. Exiting process.")
        return
    
    print("Extracting data from the downloaded Excel file...")
    df = extract_from_google_sheets(downloaded_file, SHEET_NAME)
    if df is None:
        print("Failed to extract data. Exiting process.")
        return
    
    print("Transforming data...")
    df = transform_data(df)
    
    print("Loading data into SQLite...")
    load_to_sqlite(df, DB_PATH, TABLE_NAME)
    
    print("ETL Process Completed!")

if __name__ == "__main__":
    main()

Download 100%.
File 'Take Home Test Data Engineer Baskit.xlsx' downloaded successfully to './data/'.
Extracting data from the downloaded Excel file...
Transforming data...
Loading data into SQLite...
ETL Process Completed!
